## This code is used to preprocess the data for the project

In [1]:
import nltk
import pandas as pd
from nltk.stem import WordNetLemmatizer

# Import STOPWORDS from NLTK
from nltk.corpus import stopwords

import string, re

from nltk.stem import WordNetLemmatizer

In [3]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.4 MB/s eta 0:00:00
time: 235 µs (started: 2023-04-30 05:40:48 +00:00)


In [5]:
data = pd.read_csv('https://raw.githubusercontent.com/VarshithaCVasireddy/datasets/main/dataset_new_feature.csv')

time: 784 ms (started: 2023-04-30 05:40:48 +00:00)


In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.5 MB/s eta 0:00:00
time: 18.8 s (started: 2023-04-30 05:40:52 +00:00)


In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Load the RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

model.eval()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

time: 2.99 s (started: 2023-04-28 21:28:47 +00:00)


In [ ]:
# Set the device to use for computation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the device
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

time: 375 ms (started: 2023-04-28 21:28:50 +00:00)


In [ ]:
# Define a list to store the predicted sentiment labels
predicted_sentiments = []

# Iterate over the titles in your dataset and classify them
for title in data['str1']:
    # Tokenize the title and convert it to input features for the model
    inputs = tokenizer(title, return_tensors='pt', truncation=True, padding=True)
    inputs.to('cuda')

    # Use the model to predict the sentiment label for the title
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_sentiment = torch.argmax(logits).item()

    # Map the predicted sentiment index to a string label
    predicted_sentiment_label = 'positive' if predicted_sentiment == 1 else 'negative'

    # Append the predicted sentiment label to the list
    predicted_sentiments.append(predicted_sentiment_label)

# Add a new column to the dataset containing the predicted sentiment labels
data['predicted_sentiment'] = predicted_sentiments


time: 7min 34s (started: 2023-04-28 21:28:58 +00:00)


In [9]:
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification

# Load the GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)

model.eval()
# Set the device to use for computation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the device
model.to(device)

# Define a list to store the predicted sentiment labels
predicted_sentiments_gpt2 = []

# Iterate over the titles in your dataset and classify them
for title in data['str1']:
    # Tokenize the title and convert it to input features for the model
    inputs = tokenizer(title, return_tensors='pt', truncation=True)
    inputs.to(device)

    # Use the model to predict the sentiment label for the title
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_sentiment = torch.argmax(logits).item()

    # Map the predicted sentiment index to a string label
    predicted_sentiment_label = 'positive' if predicted_sentiment == 1 else 'negative'

    # Append the predicted sentiment label to the list
    predicted_sentiments.append(predicted_sentiment_label)

# Add a new column to the dataset containing the predicted sentiment labels
data['predicted_sentiment_gpt2'] = predicted_sentiments


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


time: 9min 48s (started: 2023-04-30 05:43:18 +00:00)


In [10]:
csv_path = '/content/drive/MyDrive/sentiment_analysis_gpt2.csv'

time: 446 µs (started: 2023-04-30 05:55:11 +00:00)


In [11]:
data.to_csv(csv_path, index=False)

time: 267 ms (started: 2023-04-30 05:56:09 +00:00)


In [12]:
# Define the labels to be used for classification
labels = ["politics", "sports", "entertainment", "health", "technology", "education", "environment", "lifestyle", "crime", "religion"]


time: 551 µs (started: 2023-04-30 05:57:33 +00:00)


In [16]:
# Define a list to store the predicted labels
predicted_labels = []

# Iterate over the titles in your dataset and classify them
for title in data['str1']:
    # Tokenize the title and convert it to input features for the model
    inputs = tokenizer(title, return_tensors='pt', truncation=True)
    inputs.to(device)
    
    # Use the model to predict the label for the title
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = labels[torch.argmax(logits).item()]
    
    # Append the predicted label to the list
    predicted_labels.append(predicted_label)

# Add a new column to the dataset containing the predicted labels
data['predicted_label_gpt2'] = predicted_labels

time: 9min 44s (started: 2023-04-30 05:59:33 +00:00)


## For checking click-bait words

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/VarshithaCVasireddy/datasets/main/dataset_cleaned.csv')

time: 334 ms (started: 2023-04-28 21:16:41 +00:00)


In [23]:
import nltk
import pandas as pd
from nltk.stem import WordNetLemmatizer

# Import STOPWORDS from NLTK
from nltk.corpus import stopwords

import string, re

time: 647 µs (started: 2023-04-30 06:19:49 +00:00)


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

time: 1.25 s (started: 2023-04-28 21:16:46 +00:00)


In [ ]:
data.title = [Text.lower() for Text in data.title]
def sentence_tokenize(text: str):
    return nltk.sent_tokenize(text)

data['Sentences'] = data['title'].apply(sentence_tokenize)

time: 755 ms (started: 2023-04-28 21:16:50 +00:00)


In [ ]:
clickbait_words = {
    "shocking", "believe", "amazing", "secrets", "revealed",
    "mind-blowing", "unbelievable", "incredible", "insane", "must see",
    "astonishing", "shocking truth", "exposed", "epic", "life-changing",
    "amazing discovery", "little-known", "startling", "forbidden",
    "once-in-a-lifetime", "jaw-dropping", "bizarre", "scandalous", "sensational",
    "remarkable", "bombshell", "proof", "cover-up", "outrageous",
    "breaking", "uncovered", "conspiracy", "lies", "hoax",
    "oops", "omg", "watch", "suprise", "confirmed",
    "terrified", "terrifying", "alert", "outrage", "wow", "bam",
    "shaking", "boom", "outstanding", "awesome", "ouch", "brilliant",
    "wikileaks", "lol", "stunning", "unreal", "epic", "must read",
    "outrageous", "revealed", "explained", "exclusive"
}

time: 682 µs (started: 2023-04-28 21:16:55 +00:00)


In [ ]:
def word_tokenize(sentences: list[str]):
    words: list[str] = []
    lemmatizer = WordNetLemmatizer()
    for sent in sentences:
        for word in nltk.word_tokenize(sent):
            if word not in stopwords.words('english') and word not in string.punctuation:
                word = re.sub(r"[^a-zA-Z0-9\s.-]", "", word)
                words.append(lemmatizer.lemmatize(word))

    return words 

data['Words'] = data['Sentences'].apply(word_tokenize)

time: 58.1 s (started: 2023-04-28 21:16:57 +00:00)


In [ ]:
# create two new columns to store matched words and count
data['matched_words'] = ''
data['matched_count'] = 0

# loop through each row in the dataframe
for index, row in data.iterrows():
    # loop through each sentence in the list of sentences for the row
    for sentence in row['Sentences']:
        # use regex to find all the words in the sentence that match the clickbait words list
        matched = re.findall(r'\b(' + '|'.join(clickbait_words) + r')\b', sentence)
        # update the matched words and count columns for the row
        if len(matched) > 1:
            data.at[index, 'matched_words'] += ', '.join(matched[:-1]) + ', ' + matched[-1]
        elif len(matched) == 1:
            data.at[index, 'matched_words'] += matched[0]
        data.at[index, 'matched_count'] += len(matched)

time: 4.33 s (started: 2023-04-28 21:18:08 +00:00)


In [ ]:
data['str1'] = [' '.join(doc) for doc in data.Words]
# Create a new column 'Diff_Charac' which counts the characters in the sentence title which are not alpha numerics and space
data['Diff_Charac'] = data['title'].apply(lambda x: sum(1 for char in x if not char.isalnum() and not char.isspace()))

time: 366 ms (started: 2023-04-28 21:18:25 +00:00)
